In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


Import libraries

In [ ]:
data = pd.read_csv('fin_dataset_imputed.csv')

In [ ]:
X = data.drop(columns=['revenue', 'vote_average'])  # Features
y_revenue = data['revenue']  # Target for revenue
y_vote_average = data['vote_average']  # Target for vote average


Features and targets

In [ ]:
X_train, X_test, y_train_revenue, y_test_revenue = train_test_split(X, y_revenue, test_size=0.2, random_state=42)
X_train, X_test, y_train_vote_average, y_test_vote_average = train_test_split(X, y_vote_average, test_size=0.2, random_state=42)


Split data into training and test sets (80% train, 20% test)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Standardize the data

In [ ]:
rf_model = RandomForestRegressor(random_state=42)
gb_model = GradientBoostingRegressor(random_state=42)
svr_model = SVR()

Define base models for Voting and Stacking

In [ ]:
final_model = DecisionTreeRegressor(random_state=42)

Define the final model for stacking

In [ ]:
stacking_regressor = StackingRegressor(
    estimators=[('random_forest', rf_model), ('gradient_boosting', gb_model), ('svr', svr_model)],
    final_estimator=final_model
)

Define Stacking Regressor with Decision Tree as the final model

In [ ]:
voting_regressor = VotingRegressor(estimators=[
    ('random_forest', rf_model),
    ('gradient_boosting', gb_model),
    ('svr', svr_model)
])

Define Voting Regressor

In [ ]:
models = {
    "Random Forest": rf_model,
    "Gradient Boosting": gb_model,
    "SVR": svr_model,
    "Voting Regressor": voting_regressor,
    "Stacking Regressor": stacking_regressor
}


List models to evaluate

In [ ]:
param_grids = {
    "Random Forest": {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, 30],
        'min_samples_split': [5, 10, 20],
        'min_samples_leaf': [2, 4, 6],
        'max_features': ['sqrt', 'log2']
    },
    "Gradient Boosting": {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    },
    "SVR": {
        'C': [0.1, 1, 10],
        'epsilon': [0.01, 0.1, 0.5],
        'kernel': ['linear', 'rbf']
    }
}

Hyperparameter grids for tuning

In [ ]:
def train_and_evaluate(models, param_grids, X_train_scaled, y_train_revenue, X_test_scaled, y_test_revenue, y_train_vote_average, y_test_vote_average):
    results = {}
    
    for model_name, model in models.items():
        print(f"Training {model_name}...")
        
        if model_name not in ["Voting Regressor", "Stacking Regressor"]:  # For individual models
            grid_search_revenue = GridSearchCV(model, param_grids.get(model_name, {}), cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
            grid_search_revenue.fit(X_train_scaled, y_train_revenue)
            
            # Best model and parameters for revenue
            best_model_revenue = grid_search_revenue.best_estimator_
            best_params_revenue = grid_search_revenue.best_params_
        else:
            # For Voting and Stacking, we don't use GridSearchCV
            best_model_revenue = model
            best_params_revenue = "N/A"  # Voting and Stacking don't need GridSearch here
            best_model_revenue.fit(X_train_scaled, y_train_revenue)  # Fit the model to the training data
        
        # Predict on the test set for revenue
        y_pred_revenue = best_model_revenue.predict(X_test_scaled)
        
        # Calculate metrics for revenue
        r2_revenue = r2_score(y_test_revenue, y_pred_revenue)
        rmse_revenue = np.sqrt(mean_squared_error(y_test_revenue, y_pred_revenue))
        mae_revenue = mean_absolute_error(y_test_revenue, y_pred_revenue)
        mse_revenue = mean_squared_error(y_test_revenue, y_pred_revenue)
        
        if model_name not in ["Voting Regressor", "Stacking Regressor"]:  # For individual models
            grid_search_vote_avg = GridSearchCV(model, param_grids.get(model_name, {}), cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
            grid_search_vote_avg.fit(X_train_scaled, y_train_vote_average)
            
            # Best model and parameters for vote average
            best_model_vote_avg = grid_search_vote_avg.best_estimator_
            best_params_vote_avg = grid_search_vote_avg.best_params_
        else:
            best_model_vote_avg = model
            best_params_vote_avg = "N/A"  # Voting and Stacking don't need GridSearch here
            best_model_vote_avg.fit(X_train_scaled, y_train_vote_average)  # Fit the model to the training data
        
        # Predict on the test set for vote average
        y_pred_vote_average = best_model_vote_avg.predict(X_test_scaled)
        
        # Calculate metrics for vote average
        r2_vote_avg = r2_score(y_test_vote_average, y_pred_vote_average)
        rmse_vote_avg = np.sqrt(mean_squared_error(y_test_vote_average, y_pred_vote_average))
        mae_vote_avg = mean_absolute_error(y_test_vote_average, y_pred_vote_average)
        mse_vote_avg = mean_squared_error(y_test_vote_average, y_pred_vote_average)
        
        # Store results for both revenue and vote average models
        results[model_name] = {
            "Best Parameters for Revenue Model": best_params_revenue,
            "R² Score (Revenue)": r2_revenue,
            "RMSE (Revenue)": rmse_revenue,
            "MAE (Revenue)": mae_revenue,
            "MSE (Revenue)": mse_revenue,
            "Best Parameters for Vote Average Model": best_params_vote_avg,
            "R² Score (Vote Average)": r2_vote_avg,
            "RMSE (Vote Average)": rmse_vote_avg,
            "MAE (Vote Average)": mae_vote_avg,
            "MSE (Vote Average)": mse_vote_avg
        }
        
        # Feature importance extraction for Random Forest
        if model_name == "Random Forest":
            rf_importances = best_model_revenue.feature_importances_

            # Create a DataFrame for feature importances
            rf_importance_df = pd.DataFrame({
                'Feature': X.columns,
                'Importance': rf_importances
            })
            rf_importance_df = rf_importance_df.sort_values(by='Importance', ascending=False)

            # Plot the feature importances
            plt.figure(figsize=(12, 8))
            plt.barh(rf_importance_df['Feature'], rf_importance_df['Importance'])
            plt.xlabel('Feature Importance')
            plt.title('Random Forest Feature Importances')
            plt.show()

            print("\nRandom Forest Feature Importances:")
            print(rf_importance_df)
        
    return results

Function to train and evaluate models with GridSearchCV for both targets (revenue and vote_average) Lot of code since we need revenue and vote average for each kind of model

Now will also give insight on feature importance

In [ ]:
results = train_and_evaluate(models, param_grids, X_train_scaled, y_train_revenue, X_test_scaled, y_test_revenue, y_train_vote_average, y_test_vote_average)

Run the training and evaluation

In [ ]:
for model_name, metrics in results.items():
    print(f"\n{model_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

Print the results for all models